## 가위바위보_구별기
-----

### 반복 사용되는 모듈 임포트 

In [174]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os, glob, cv2


print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


### 가위, 바위, 보 이미지 전처리

In [103]:
# test와 train 모두 처리하는 함수
def resize_images(img_path): 
    for folder in os.listdir(img_path):
        if folder not in ['rock', 'scissor', 'paper']: 
            continue   
        images=glob.glob(img_path + f"/{folder}/*.jpg")  
        print(f"{len(images)} {folder} images to be resized.")
        # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
        target_size=(28,28)
        for img in images:
            old_img=cv2.imread(img,cv2.IMREAD_GRAYSCALE) # 사진을 흑백으로 변환한다.
            new_img=cv2.resize(old_img,target_size,Image.ANTIALIAS)
            cv2.imwrite(img,new_img)
        print(f"{len(images)} {folder} images resized.")

In [104]:
folder_name = input("test/train 중 하나를 입력하세요")
image_dir_path = os.getenv("HOME") + f"/aiffel/rock_scissor_paper/{folder_name}"
resize_images(image_dir_path)

print(f"{folder_name} 이미지 resize 완료!")

test/train 중 하나를 입력하세요test
92 paper images to be resized.
92 paper images resized.
77 rock images to be resized.
77 rock images resized.
66 scissor images to be resized.
66 scissor images resized.
test 이미지 resize 완료!


### 전처리된 이미지 행렬화 및 정규화

In [175]:
# test와 train 모두 처리하는 함수
def load_data(img_path, number_of_data=300): 
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=1
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size, color)
    labels=np.zeros(number_of_data,dtype=np.int32)
     
    # number_of_data를 3분의 1로 나눠 가위/바위/보를 균등하게 채움   
    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        if idx > (number_of_data//3)-1: # 가위 이미지의 개수상한 초과시 추가하지 않음.
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
        
    for file in glob.iglob(img_path+'/rock/*.jpg'):
        if idx > (number_of_data//3*2)-1: # 바위 이미지의 개수상한 초과시 추가하지 않음.
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  

    for file in glob.iglob(img_path+'/paper/*.jpg'):
        if idx > number_of_data-1:  # 보 이미지의 개수상한 초과시 추가하지 않음.
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print(f"처리된 데이터의 이미지 개수는 {idx}개 입니다.")
    return imgs, labels

In [135]:
folder_name = input("train/test 중 하나를 입력하세요")
image_dir_path = os.getenv("HOME") + f"/aiffel/rock_scissor_paper/{folder_name}"
if folder_name == "train":
    (x_train, y_train)=load_data(image_dir_path,3300) # 최대 3300개의 학습데이터를 볼러올 수 잇다. 
    x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화
    x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 정규화된 이미지 reshape
    print(f"x_train shape: {x_train_reshaped.shape}")
    print(f"y_train shape: {y_train.shape}")
else: 
    (x_test, y_test)=load_data(image_dir_path) # 기본값인 300개의 학습데이터를 볼러올 수 잇다. 
    x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화
    x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 정규화된 이미지 reshape
    x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)
    print(f"x_test reshaped: {x_test_reshaped.shape}")
    print(f"y_test reshaped: {y_test.shape}")

train/test 중 하나를 입력하세요train
처리된 데이터의 이미지 개수는 3207개 입니다.
x_train shape: (3300, 28, 28, 1)
y_train shape: (3300,)


* train 데이터는 AIFFEL 동료들의 가위바위보이미지 3207장을 사용하였다. 
* 

### 학습 모델만들기  

[컨볼루션 신경망 레이어 이야기](https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/)를 참조하여 실습을 진행했다

In [171]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Flatten, Dense

# Sequential 모델 불러오기
model = Sequential()
#  28X28X1의 입력이미지를 입력받는 4X4 크기의 필터 16개가 탑재된 Conv2D레이어 추가  
model.add(Conv2D(16, (4,4), activation='relu', input_shape=(28,28,1)))
# Conv2D레이어의 출력이미지를 입력받아 2X2사이즈 이미지로 변환해 출력하는 MaxPool2D 레이어 추가
model.add(MaxPool2D(2,2))
model.add(Conv2D(16, (4,4), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2))) # MaxPool2D = MaxPooling2D
# 이미지를 1차원으로 바꾸어 주는 Flatton 레이어 추가
model.add(Flatten())
# 입력뉴런 4개 출력뉴런 64개인 Dense레이어 추가           
model.add(Dense(64, activation='relu', input_dim=4))
# 출력뉴런 16개인 Dense레이어를 model에 추가   
model.add(Dense(16, activation='softmax'))

model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 25, 25, 16)        272       
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 9, 9, 16)          4112      
_________________________________________________________________
max_pooling2d_51 (MaxPooling (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_48 (Dense)             (None, 16)              

### 학습

In [172]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델이 20번 학습한다.                                      
model.fit(x_train_reshaped, y_train, epochs=20)

Epoch 1/20
104/104 [==============================] - 1s 3ms/step - loss: 1.6504 - accuracy: 0.3328
Epoch 2/20
104/104 [==============================] - 0s 3ms/step - loss: 1.0567 - accuracy: 0.4137
Epoch 3/20
104/104 [==============================] - 0s 3ms/step - loss: 0.9989 - accuracy: 0.4548
Epoch 4/20
104/104 [==============================] - 0s 3ms/step - loss: 0.9050 - accuracy: 0.5711
Epoch 5/20
104/104 [==============================] - 0s 3ms/step - loss: 0.8176 - accuracy: 0.6124
Epoch 6/20
104/104 [==============================] - 0s 3ms/step - loss: 0.7784 - accuracy: 0.6291
Epoch 7/20
104/104 [==============================] - 0s 3ms/step - loss: 0.6987 - accuracy: 0.6941
Epoch 8/20
104/104 [==============================] - 0s 3ms/step - loss: 0.6238 - accuracy: 0.7409
Epoch 9/20
104/104 [==============================] - 0s 3ms/step - loss: 0.5564 - accuracy: 0.7572
Epoch 10/20
104/104 [==============================] - 0s 3ms/step - loss: 0.5054 - accuracy: 0.7982

### 테스트

In [173]:
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print(f'test_loss: {test_loss}')
print(f'test_accuracy: {test_accuracy}')

10/10 - 0s - loss: 1.3219 - accuracy: 0.6967
test_loss: 1.321900725364685
test_accuracy: 0.6966666579246521


## 후기
-----
이번 과제를 수행하는 동안 어려웠던 점은 역시 **accuracy 60을 넘기는 것**이었다.

오버피팅을 극복하기 위해 학습데이터의 크기를 처음 300장에서 3207장까지 늘리고 이미지를 그레이스케일로 변환하고 여러 시도를 했었다.

그러다보니 99, 80, 67점 이렇게 3번이나 60을 넘겨보았지만, 

참지못하고 다시 시도하는 바람에 마지막 69점을 만들기까지 긴 시간이 걸렸다.

하지만 그러면서 여러가지 시도를 하다보니 배우게 된 부분 역시 많았다.

<br>

#### 1. cv2 모듈을 활용한 이미지파일 그레이스케일 처리

grayscale적용 자체는 검색으로 어렵지 않게 했지만, plt로 출력할 때 색상이 예상밖의 색으로 나와 당황스러웠다.

lms조원들에게 물어보니 plt.imshow의 인자로 cmap='gray'를 넘겨줌으로써 출력시에도 흑백으로 설정해주면 되는 문제였다.

모델 학습에는 지장이 없는 문제였으나 한동안 풀지 못했던 기억이 난다.


#### 2. [keras의 sequential 모델과 그 레이어들의 사용법](https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/)

위 링크의 글을 많이 참조하였는데, 학습모델생성시 사용되는 sequential 모델과 그 아래의 Conv2D, maxPooling2D, Faltton, Dense레이어들에 대한 사용법과 sequential모델의 전체 그림까지 설명이 잘되어 있어서 좋았다. 이 글을 참조하며 레이어들의 파라미터를 조절하다보니 69%까지 다시 도달할 수 있었다. 하지만 아직 컨볼루션 신경망 레이어의 구조가 완전히 잘 이해되지 않아 아쉬웠다.

#### 3. 웹캠 영상 프레임 단위로 이미지 저장 코드 활용
test 데이터를 생성할 때 lms조원이 구글링 하여 공유해준 웹캠 영상을 프레임 단위로 이미지 파일로 자동저장하는 아래의 스크립트를 활용해보았다. 

```python
import cv2

# open webcam (웹캠 열기)
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()

sample_num = 0
captured_num = 0

# loop through frames
while webcam.isOpened():

    # read frame from webcam
    status, frame = webcam.read()
    sample_num = sample_num + 1

    if not status:
        break

    # display output
    cv2.imshow("captured frames", frame)

    if sample_num == 4:
        captured_num = captured_num + 1
        cv2.imwrite('C:/Users/PC/Pictures/RPS_images/rock/img' + str(captured_num) + '.jpg', frame)  # 바위 이미지 수집시
        # cv2.imwrite('C:/Users/PC/Pictures/RPS_images/paper/img'+str(captured_num)+'.jpg', frame) # 보 이미지 수집시
        # cv2.imwrite('C:/Users/PC/Pictures/RPS_images/scissors/img'+str(captured_num)+'.jpg', frame) # 가위 이미지 수집시
        sample_num = 0

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()
```

출처: [[Anaconda+python] 웹캠 영상 프레임 샘플링해서 저장하기(쉽게 이미지 데이터베이스 만들기).txt](https://bskyvision.com/723)

코드자체는 가위/바위/보의 경로가 바뀌는 것을 수동으로 주석을 해제/설정 하는 것으로 스위칭하는 것 외엔 큰 문제가 없었으나, gather에서 cam을 사용할 경우 cam을 사용하지 못해 에러가 발생하는 문제가 있어서 잠시 캠을 끄고 이미지를 만들었던 기억이 난다. 급하게 이미지를 대량으로 만들어야할 필요가 있을 때 쓰면 가성비가 좋을 것 같다. 


#### 마무리
이번 노드를 진행하면서 처음 머신러닝을 경험해볼 수 있어서 재미있게 노드를 진행했었다. 하지만 그와 동시에 내가 뭘 모르고 있는지 다시 느낄 수 있었다. 특히 구글링으로 찾아낸 블로그 글에서 기술한 컨볼루션 신경망 레이어의 구조를 보면서 내가 앞으로 배워야할 것이 이런 것이구나 하고 처음 제대로 알게된 기분이들었다. 지금은 낯설기만 한 것이지만 6개월이 지난 후의 나에게는 익숙한 것이 되어있었으면 좋겠고, 꼭 그렇게 만들고 싶다.    